***负面评论识别：用word2vec，及doc2vec提取特征
   通过word2vec/doc2vec,我们既可以获取一维特征向量，也可以获取二维的特征向量

In [5]:
"""
第一步：导入相关的包

这里我们换安装了一个较低版本的gensim，就没有报错了
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing


max_features=200
max_document_length=500
vocabulary=None
doc2ver_bin="doc23ver.bin"
word2ver_bin="word23ver.bin"
#LabeledSentence = gensim.models.doc2vec.LabeledSentence
SentimentDocument = namedtuple('SentimentDocument', 'words tags')

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-17 18:39:58
end_time: 2021-11-17 18:39:58


In [6]:
"""
第二步：读取文件数据,

看起来像是将一个文档(文件)加载到一个字符串,然后再保存到列表中。
"""
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

def load_one_file(filename):
    x=""
    with open(filename,encoding='gb18030', errors='ignore') as f:
        for line in f:
            line=line.strip('\n')
            line = line.strip('\r')
            x+=line
    f.close()
    return x

def load_files_from_dir(rootdir):
    x=[]
    list = os.listdir(rootdir)
    for i in range(0, len(list)):
        path = os.path.join(rootdir, list[i])
        if os.path.isfile(path):
            v=load_one_file(path)
            x.append(v)
    return x

def load_all_files():
    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    path="E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/"
    print ("Load %s" % path)
    x_train=load_files_from_dir(path)
    y_train=[0]*len(x_train)
    path="E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/"
    print ("Load %s" % path)
    tmp=load_files_from_dir(path)
    y_train+=[1]*len(tmp)
    x_train+=tmp

    path="E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/"
    print ("Load %s" % path)
    x_test=load_files_from_dir(path)
    y_test=[0]*len(x_test)
    path="E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/"
    print ("Load %s" % path)
    tmp=load_files_from_dir(path)
    y_test+=[1]*len(tmp)
    x_test+=tmp

    return x_train, x_test, y_train, y_test

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-17 18:39:59
end_time: 2021-11-17 18:39:59


In [12]:
"""
第三步：定义训练模型
分为机器学习与深度学习模型两部分
"""
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

#下面的是机器学习模型
def do_svm_doc2vec(x_train, x_test, y_train, y_test):
    #print ("SVM and doc2vec")
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print (metrics.accuracy_score(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))
    
def do_nb_doc2vec(x_train, x_test, y_train, y_test):
    #print ("NB and doc2vec")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print (metrics.accuracy_score(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))
    
def do_rf_doc2vec(x_train, x_test, y_train, y_test):
    #print ("rf and doc2vec")
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print (metrics.accuracy_score(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))
    
   
#下面的是深度学习模型  
#卷积神经网络二元
def do_cnn_doc2vec_2d(trainX, testX, trainY, testY):
    #print ("CNN and doc2vec 2d")

    trainX = trainX.reshape([-1, max_features, max_document_length, 1])
    testX = testX.reshape([-1, max_features, max_document_length, 1])
    #对y处理得这段代码自己添加的。
    y_train_one_hot = []
    y_test_one_hot = []
    for i in range(len(trainY)):
        if ((i+1)%500==0 and trainY[i]== 0):
            y_train_one_hot.append([float(1),float(0)])
        elif ((i+1)%500==0 and trainY[i] == 1):
            y_train_one_hot.append([float(0),float(1)])  
        
    for i in range(len(testY)):
        if ((i+1)%500==0 and testY[i]== 0):
            y_test_one_hot.append([float(1),float(0)])
        elif ((i+1)%500==0 and testY[i] == 1):
            y_test_one_hot.append([float(0),float(1)])  
        
    trainY = np.array(y_train_one_hot)#
    testY = np.array(y_test_one_hot)


    # Building convolutional network
    network = input_data(shape=[None, max_features, max_document_length, 1], name='input')
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = fully_connected(network, 128, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='tanh')
    network = dropout(network, 0.8)
    #network = fully_connected(network, 10, activation='softmax')
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit({'input': trainX}, {'target': trainY}, n_epoch=20,
               validation_set=({'input': testX}, {'target': testY}),
               snapshot_step=100, show_metric=True, run_id='review')

    #卷积神经网络：一元
def do_cnn_doc2vec(trainX, testX, trainY, testY):
    global max_features
    #print ("CNN and doc2vec")

    #trainX = pad_sequences(trainX, maxlen=max_features, value=0.)
    #testX = pad_sequences(testX, maxlen=max_features, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_features], name='input')  #注意这里的维度是一维向量
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128,validate_indices=False)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="review")


#多层感知机模型(最简单的全连接神经网络)
def do_dnn_doc2vec(x_train, x_test, y_train, y_test):
    #print ("MLP and doc2vec")
    global max_features
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    print  (clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print (metrics.accuracy_score(y_test, y_pred))
    print (metrics.confusion_matrix(y_test, y_pred))

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-17 18:41:21
end_time: 2021-11-17 18:41:21


In [13]:

"""
第四步：获取word2vec/doc2vec文本特征的预处理工作
"""
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

#对特殊符号与标点符号做处理
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n', '') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    # treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s ' % c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides 在标点符号的两侧加空格
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text

#doc2vec需要给每个reviewoc2Vec处理的每个英文段落，需要使用一个唯一的标识来标记
def labelizeReviews(reviews, label_type):
    labelized = []
    for i, v in enumerate(reviews):
        label = '%s_%s' % (label_type, i)
        #labelized.append(LabeledSentence(v, [label]))
        #labelized.append(LabeledSentence(words=v,tags=label))
        labelized.append(SentimentDocument(v, [label]))
    return labelized

def getVecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]]).reshape((1, size)) for z in corpus]
    return np.array(np.concatenate(vecs),dtype='float')


def buildWordVector(imdb_w2v,text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-17 18:41:24
end_time: 2021-11-17 18:41:24


In [5]:
"""
第五步：获取文本特征
       开始训练word2vec模型
       取值存在负数
"""
word2ver_bin = "review_word2vec.bin"

print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
#word2vec文本特征模型会将每一个词语转化为n维向量，所以内存会不够
def  get_features_by_word2vec():
    global  max_features
    global word2ver_bin
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x=x_train+x_test
    cores=multiprocessing.cpu_count() #获取当前计算机的cpu个数

    if os.path.exists(word2ver_bin):
        print ("Find cache file %s" % word2ver_bin)
        model=gensim.models.Word2Vec.load(word2ver_bin)
    else:
        model=gensim.models.Word2Vec(size=max_features, window=10, min_count=1, iter=60, workers=cores)

        model.build_vocab(x)

        model.train(x, total_examples=model.corpus_count, epochs=model.iter)
        model.save(word2ver_bin)


    x_train= np.concatenate([buildWordVector(model,z, max_features) for z in x_train])
    x_train = scale(x_train)
    x_test= np.concatenate([buildWordVector(model,z, max_features) for z in x_test])
    x_test = scale(x_test)

    return x_train, x_test, y_train, y_test

print ("get_features_by_word2vec")
x_train, x_test, y_train, y_test=get_features_by_word2vec()
print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
x_train[1]


start_time: 2021-11-12 15:51:16
get_features_by_word2vec
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_word2vec.bin
end_time: 2021-11-12 15:55:00


array([-3.93052335e-01, -6.18693679e-01, -3.66564747e-01,  1.94379200e+00,
        5.17187478e-02,  1.77266765e+00, -1.13460904e+00, -3.13377797e-01,
       -2.10513102e-03, -6.58536136e-01, -2.33050575e+00, -1.01432710e+00,
        8.28339694e-01,  2.11758228e+00,  1.28951850e+00, -6.75243553e-01,
       -1.50628626e+00,  1.99503177e+00,  1.65627989e-01,  6.16613498e-01,
        4.73955577e-01, -1.25254573e-01, -1.11096287e+00, -5.07561825e-01,
        2.00181772e-01, -1.72721201e-01, -5.33310333e-01, -1.74073466e+00,
       -1.83294122e-02, -1.83414718e+00, -2.01485962e+00,  1.43616146e+00,
       -5.00532050e-01, -1.03555173e+00, -2.40348199e-01,  1.39355964e-01,
       -3.89994850e-01,  6.15300557e-01,  1.37428103e+00, -1.28621961e+00,
        5.47313351e-02,  9.20719203e-01, -6.83533426e-02,  1.74424609e+00,
       -1.22915304e-01,  8.18166474e-01, -6.12105775e-01,  1.64017432e+00,
       -4.40652735e-01, -4.29670367e-01, -4.86410967e-01, -4.90008247e-01,
        6.27448884e-01, -

In [6]:
"""
第五步：获取文本特征
       开始训练word2vec_cnn1d模型
       取值不存在负数,可以用CNN模型，唯一的区别就是上个单元格word2vec对x_train进行标准化处理，这里是归一化处理,
"""
word2ver_bin_cnn1d = "review_word2vec_cnn1d.bin"

print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
#word2vec文本特征模型会将每一个词语转化为n维向量，所以内存会不够
def  get_features_by_word2vec_cnn_1d():
    global  max_features
    global word2ver_bin_cnn1d
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x=x_train+x_test
    cores=multiprocessing.cpu_count() #获取当前计算机的cpu个数

    if os.path.exists(word2ver_bin_cnn1d):
        print ("Find cache file %s" % word2ver_bin_cnn1d)
        model=gensim.models.Word2Vec.load(word2ver_bin_cnn1d)
    else:
        model=gensim.models.Word2Vec(size=max_features, window=10, min_count=1, iter=60, workers=cores)

        model.build_vocab(x)

        model.train(x, total_examples=model.corpus_count, epochs=model.iter)
        model.save(word2ver_bin_cnn1d)

    min_max_scaler = preprocessing.MinMaxScaler()
    x_train= np.concatenate([buildWordVector(model,z, max_features) for z in x_train])
    #x_train = scale(x_train) #这里标准化处理数据介于[-1,1],但是CNN模型是不能负数？
    x_train = min_max_scaler.fit_transform(x_train)
    x_test= np.concatenate([buildWordVector(model,z, max_features) for z in x_test])
    #x_test = scale(x_test)
    x_test = min_max_scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

print ("get_features_by_word2vec_cnn_1d")
x_train, x_test, y_train, y_test=get_features_by_word2vec_cnn_1d()
print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
x_train[1]


start_time: 2021-11-12 15:55:00
get_features_by_word2vec_cnn_1d
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_word2vec_cnn1d.bin
end_time: 2021-11-12 15:55:47


array([0.39258288, 0.38558334, 0.35173874, 0.76502889, 0.28214877,
       0.55568782, 0.63172517, 0.1241633 , 0.30440861, 0.31939899,
       0.31816324, 0.49371318, 0.5394323 , 0.7197188 , 0.15799385,
       0.31364137, 0.30582918, 0.43460088, 0.40009477, 0.75454824,
       0.55829589, 0.30142037, 0.40112753, 0.30778506, 0.80458636,
       0.4512612 , 0.50565783, 0.21746247, 0.7071669 , 0.2708229 ,
       0.4270079 , 0.63145101, 0.27639013, 0.39352361, 0.77392144,
       0.60174232, 0.44531788, 0.35229482, 0.47669598, 0.70511187,
       0.71569787, 0.65529092, 0.73051261, 0.6434533 , 0.72184362,
       0.42177348, 0.6172958 , 0.41333111, 0.37764418, 0.34879563,
       0.21575099, 0.24806032, 0.35082055, 0.71397961, 0.26428397,
       0.66022163, 0.37270537, 0.4946965 , 0.72343404, 0.57885381,
       0.47149924, 0.61474078, 0.61860526, 0.36730958, 0.47656038,
       0.45561113, 0.56377503, 0.71573091, 0.30952602, 0.55439629,
       0.55267237, 0.42531454, 0.30238702, 0.40555068, 0.44835

In [9]:
"""
第五步：获取文本特征
       开始训练doc2vec模型
"""
doc2ver_bin = "review_doc2vec.bin"
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
def  get_features_by_doc2vec():
    global  max_features
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)
    
    #这里的x_train作为输入不仅包括review，还包括段落id
    x_train = labelizeReviews(x_train, 'TRAIN')
    x_test = labelizeReviews(x_test, 'TEST')

    x=x_train+x_test
    cores=multiprocessing.cpu_count()
    #models = [
        # PV-DBOW
    #    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=19, iter=10, workers=cores),
        # PV-DM w/average
    #    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=19, iter=10, workers=cores),
    #]
    if os.path.exists(doc2ver_bin):
        print ("Find cache file %s" % doc2ver_bin)
        model=Doc2Vec.load(doc2ver_bin)
    else:
        model=Doc2Vec(dm=0, size=max_features, negative=5, hs=0, min_count=2, workers=cores,iter=60)
        model.build_vocab(x)
        model.train(x, total_examples=model.corpus_count, epochs=model.iter)
        model.save(doc2ver_bin)
    
    x_test=getVecs(model,x_test,max_features)
    x_train=getVecs(model,x_train,max_features)
    min_max_scaler = preprocessing.MinMaxScaler() #这里我们自己进行归一化处理，看看数据有没有用
    
    x_train = min_max_scaler.fit_transform(x_train)
    x_test = min_max_scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

print ("get_features_by_doc2vec")
x_train, x_test, y_train, y_test=get_features_by_doc2vec()

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
x_train[1]

start_time: 2021-11-17 18:40:10
get_features_by_doc2vec
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_doc2vec.bin
end_time: 2021-11-17 18:40:20


array([0.44783841, 0.42543603, 0.5684536 , 0.58249367, 0.31805432,
       0.69776262, 0.43147905, 0.49470703, 0.54687621, 0.28639542,
       0.46503794, 0.46804945, 0.51905424, 0.69251359, 0.41303784,
       0.48299472, 0.61325353, 0.55412321, 0.66355049, 0.47657671,
       0.49486381, 0.53723719, 0.56501481, 0.6408539 , 0.49864768,
       0.67933776, 0.88649434, 0.28501082, 0.37121625, 0.5406572 ,
       0.48228556, 0.56659812, 0.65961875, 0.47979771, 0.32683184,
       0.37223713, 0.44664571, 0.62299622, 0.4392753 , 0.47715828,
       0.53552034, 0.49536691, 0.46754355, 0.3941373 , 0.33787561,
       0.56577254, 0.49515422, 0.57381413, 0.33702676, 0.24513948,
       0.51113941, 0.4694651 , 0.49018001, 0.27253831, 0.41425049,
       0.44726507, 0.58939833, 0.59209769, 0.59959119, 0.49908834,
       0.32583658, 0.42472888, 0.40495883, 0.48870094, 0.28329501,
       0.4072202 , 0.47014851, 0.54588969, 0.41900231, 0.59159348,
       0.38075823, 0.5860117 , 0.5763693 , 0.5809582 , 0.67620

In [30]:
"""
第六步:模型训练：
基于word2vec提取的文本特征1
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

print ("start get_features_by_word2vec--->")
x_train, x_test, y_train, y_test=get_features_by_word2vec()

print("start SVM")
do_svm_doc2vec(x_train, x_test, y_train, y_test)

print("start NB")
do_nb_doc2vec(x_train, x_test, y_train, y_test)

print("start RF")
do_rf_doc2vec(x_train, x_test, y_train, y_test)

print("start DNN")
do_dnn_doc2vec(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-11 23:44:40
start get_features_by_word2vec--->
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_word2vec.bin
start SVM
0.87916
[[11017  1483]
 [ 1538 10962]]
start NB
0.6714
[[7829 4671]
 [3544 8956]]
start RF
0.74964
[[10221  2279]
 [ 3980  8520]]
start DNN
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
0.87048
[[10838  1662]
 [ 1576 10924]]
end_time: 2021-11-11 23:49:43


D:\anaconda3_5\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [7]:
"""
第六步:模型训练：
基于word2vec_cnn_1d提取的文本特征1
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

print ("start get_features_by_word2vec_cnn_1d--->")
x_train, x_test, y_train, y_test=get_features_by_word2vec_cnn_1d()

print("start SVM")
do_svm_doc2vec(x_train, x_test, y_train, y_test)

print("start NB")
do_nb_doc2vec(x_train, x_test, y_train, y_test)

print("start RF")
do_rf_doc2vec(x_train, x_test, y_train, y_test)

print("start DNN")
do_dnn_doc2vec(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-12 16:13:22
start get_features_by_word2vec_cnn_1d--->
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_word2vec_cnn1d.bin
start SVM
0.87864
[[10970  1530]
 [ 1504 10996]]
start NB
0.67616
[[7748 4752]
 [3344 9156]]
start RF
0.74616
[[10115  2385]
 [ 3961  8539]]
start DNN
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
0.5
[[12500     0]
 [12500     0]]
end_time: 2021-11-12 16:15:29


In [8]:
"""
第六步:模型训练：
基于word2vec_cnn_1d提取的文本特征2
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))
print ("start get_features_by_word2vec_cnn_1d--->")
x_train, x_test, y_train, y_test=get_features_by_word2vec_cnn_1d()

print("start word2vec_CNN_1d")
do_cnn_doc2vec(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

Training Step: 1249  | total loss: 0.69319 | time: 251.676s
| Adam | epoch: 005 | loss: 0.69319 - acc: 0.4956 -- iter: 24900/25000
Training Step: 1250  | total loss: 0.69318 | time: 264.915s
| Adam | epoch: 005 | loss: 0.69318 - acc: 0.4991 | val_loss: 0.69315 - val_acc: 0.5000 -- iter: 25000/25000
--
end_time: 2021-11-12 16:41:42


In [41]:
"""
第六步:模型训练：
基于doc2vec提取的文本特征1
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

print ("start get_features_by_doc2vec--->")
x_train, x_test, y_train, y_test=get_features_by_doc2vec()

print("start SVM")
do_svm_doc2vec(x_train, x_test, y_train, y_test)

print("start NB")
do_nb_doc2vec(x_train, x_test, y_train, y_test)

print("start RF")
do_rf_doc2vec(x_train, x_test, y_train, y_test)

print("start DNN")
do_dnn_doc2vec(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-13 14:28:31
start get_features_by_doc2vec--->
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_doc2vec.bin
start SVM
0.87628
[[10714  1786]
 [ 1307 11193]]
start NB
0.83764
[[10151  2349]
 [ 1710 10790]]
start RF
0.67592
[[9420 3080]
 [5022 7478]]
start DNN
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
0.5
[[12500     0]
 [12500     0]]
end_time: 2021-11-13 14:33:26


In [6]:
"""
第六步:模型训练：
基于doc2vec提取的文本特征2:cnn_1d
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

print ("start get_features_by_doc2vec--->")
x_train, x_test, y_train, y_test=get_features_by_doc2vec()

print("start doc2vec_CNN")
do_cnn_doc2vec(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

Training Step: 1249  | total loss: 0.69319 | time: 258.424s
| Adam | epoch: 005 | loss: 0.69319 - acc: 0.4976 -- iter: 24900/25000
Training Step: 1250  | total loss: 0.69322 | time: 271.597s
| Adam | epoch: 005 | loss: 0.69322 - acc: 0.4938 | val_loss: 0.69316 - val_acc: 0.5000 -- iter: 25000/25000
--
end_time: 2021-11-13 15:19:10


In [14]:
"""
第六步:模型训练：
基于doc2vec提取的文本特征2:cnn_2d
trainX.shape = (50, 200, 500, 1)
这里得CNN_2d我们是进行了处理的,需要进一步对y进行维度变化，以及one_hot编码,(确保进行模型训练的数据格式正确)
"""
import datetime
print('start_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

print ("start get_features_by_doc2vec--->")
x_train, x_test, y_train, y_test=get_features_by_doc2vec()

print("start doc2vec_CNN_2D")
do_cnn_doc2vec_2d(x_train, x_test, y_train, y_test)

print('end_time: ' + datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S'))

start_time: 2021-11-17 18:41:34
start get_features_by_doc2vec--->
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/train/neg/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/pos/
Load E:/pycharm_project/deep_learning_web_safe/review/data/test/neg/
Find cache file review_doc2vec.bin
start doc2vec_CNN_2D
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
-----------------------------